In [1]:
import re
import pickle
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

from _html_parser import HtmlParser

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('pickle_files/teams_data/df_game_ids_na_cities', 'rb') as f:
    df_game_ids_na_cities = pickle.load(f)

In [4]:
df_game_ids_na_cities.reset_index(drop=True, inplace=True)

In [5]:
df_game_ids_na_cities.tail(3)

,game_id,f_com
1338,10917426,SønderjyskE
1339,10915588,Neftekhimik
1340,10639504,Tauras


In [6]:
str_inds = [str(x) for x in df_game_ids_na_cities.game_id]
str_inds[:2]

['1459330', '1459333']

In [7]:
# HtmlParser.find_teams_ids(str_inds[1001:])

In [8]:
with open('pickle_files/teams_data/teams_ids_na_cities_14198781', 'rb') as f:
    teams_ids_1 = pickle.load(f)

In [9]:
with open('pickle_files/teams_data/teams_ids_na_cities_10639504', 'rb') as f:
    teams_ids_2 = pickle.load(f)

In [10]:
teams_ids = teams_ids_1 + teams_ids_2

In [11]:
df_teams_ids = pd.DataFrame(teams_ids, columns=['team_id', 'team'])
df_teams_ids.reset_index(inplace=True)
df_teams_ids.drop_duplicates(subset='team', keep='first', inplace=True)

In [12]:
tm_ids = df_teams_ids.team_id.values

In [13]:
# HtmlParser.find_teams_data(tm_ids)

In [14]:
with open('pickle_files/teams_data/teams_wiki_data_7506', 'rb') as f:
    wiki_data = pickle.load(f)

In [15]:
wiki_data[:1]

[('12433',
  'FC Polissya Zhytomyr  is a Ukrainian football club based in Zhytomyr, Polesia. As of the 2020–21 season, it will play in the Ukrainian First League, the second tier of Ukrainian football, following promotion from the 2019–20 Ukrainian Second League. Their home were Spartak and Kolos stadiums.')]

In [141]:
df_wiki_data = pd.DataFrame(wiki_data, columns=['team_id', 'tm_data'])
df_wiki_data = df_wiki_data[df_wiki_data.index.isin(df_teams_ids.index)].copy()
df_teams_data = df_teams_ids.merge(df_wiki_data, how='left')
df_teams_data = df_teams_data[df_teams_data.tm_data.values != ''].copy()
df_teams_data.drop_duplicates(subset='tm_data', keep='first', inplace=True)
df_teams_data.index = df_teams_data['index'].values
df_teams_data.drop(columns=['index'], inplace=True)
df_teams_data = df_teams_data.merge(df_game_ids_na_cities.game_id, how='left', left_index=True, right_index=True)

In [142]:
print(len(df_teams_data))
df_teams_data.tail(5)

1064


,team_id,team,tm_data,game_id
1334,7233,Paulista,"Paulista Futebol Clube, or Paulista as they are usually called, is a Brazilian football team from Jundiaí, in São Paulo, founded on 17 May 1909. They currently play in the Paulistão Segunda Divisão, the fourth tier of the São Paulo state league.",10862479
1336,7656,Pacífico,"Club Deportivo Pacífico FC is a Peruvian football club. Based in the San Martín de Porres District, in the department of Lima, it plays in Peru's Second Division.",10633613
1337,7920,Guaratinguetá,"Guaratinguetá Futebol Ltda., usually known simply as Guaratinguetá, was a Brazilian football club from Guaratinguetá, São Paulo state.",10625571
1338,7325,SønderjyskE,"SønderjyskE is a Danish sport corporation. They have a football team (SønderjyskE Fodbold, based in Haderslev), two handball teams (SønderjyskE Håndbold, one male based in Sønderborg, one female based in Aabenraa) and an ice hockey team (SønderjyskE Ishockey, based in Vojens).",10917426
1340,7506,Tauras,FK Tauras is a Lithuanian football team from the city of Tauragė.,10639504


In [82]:
df_teams_na_data = df_game_ids_na_cities[~df_game_ids_na_cities.index.isin(df_teams_data.index.values)].copy()

In [83]:
df_teams_na_data.shape

(277, 2)

In [63]:
data_is_a = df_teams_data.tm_data.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'is a', '\.')) if re.findall('is a', str(x)) else None)

In [37]:
data_based_in = data_is_a.map(lambda x: x if re.findall('(based in)|(city of)|(town of)|(located in)|(club from)|(club in)|(playing in)', str(x)) else None)

In [23]:
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'city of ', '$')) if re.findall('city of', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'town of ', '$')) if re.findall('town of', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'located in ', '$')) if re.findall('located in', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'club from ', '$')) if re.findall('club from', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'club in ', '$')) if re.findall('club in', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'playing in ', '$')) if re.findall('playing in', str(x)) else x)
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'based in ', '$')) if re.findall('based in', str(x)) else x)

In [24]:
data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, '^', ',')) if re.findall(',', str(x)) else x)

In [72]:
data_is_a = data_is_a.map(lambda x: re.sub(r'^n ', '', str(x)))
data_is_a = data_is_a.map(lambda x: re.sub(r'^ ', '', str(x)))

In [138]:
country = data_is_a.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, '^', ' ')))

In [139]:
country = country.map(lambda x: re.sub('professional', '', str(x)))
country = country.map(lambda x: re.sub('football', '', str(x)))

In [140]:
df_country = country.map(lambda x: re.sub(r'n$', '', str(x))).to_frame(name='country')

In [143]:
df_teams_data = df_teams_data.merge(df_based_in, how='left', left_index=True, right_index=True)
df_teams_data = df_teams_data.merge(df_country, how='left', left_index=True, right_index=True)

In [145]:
df_teams_data.head(3)

,team_id,team,tm_data,game_id,city,country
0,12433,Polessya,"FC Polissya Zhytomyr is a Ukrainian football club based in Zhytomyr, Polesia. As of the 2020–21 season, it will play in the Ukrainian First League, the second tier of Ukrainian football, following promotion from the 2019–20 Ukrainian Second League. Their home were Spartak and Kolos stadiums.",1459330,Zhytomyr,Ukrainia
1,7156,Kramatorsk,Football Club Kramatorsk is a professional Ukrainian football club from the city of Kramatorsk in Donetsk Oblast. The club takes its history from a factory team of the Old Kramatorsk Machine-building Plant (SKMZ).,1459333,Kramatorsk,
2,8374,Lincoln City,"Lincoln City Football Club is a professional association football club based in the city of Lincoln, Lincolnshire, England. The team compete in EFL League One, the third tier of the English football league system.",1445410,Lincoln,


In [148]:
df_teams_data.country.value_counts(dropna=False).head(4)

             280
Italia        71
Brazilia      63
Argentine     44
Name: country, dtype: int64

In [150]:
# file = open('pickle_files/teams_data/df_teams_data', 'wb')
# pickle.dump(df_teams_data, file)  
# file.close()

In [153]:
# with open('pickle_files/teams_data/df_teams_data', 'rb') as f:
#     df_tm_data = pickle.load(f)

In [155]:
# df_tm_data.to_csv('csv_files/df_teams_data.csv', index = None, header=True)